In [425]:
import os
import random
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import tqdm

In [426]:
data_path = "data"

seed=12345
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
device=torch.device("cuda" if torch.cuda.is_available() else "mps")
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Define a simple CNN:

In [427]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.convolutional_layer = nn.Sequential(            
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
                        
            nn.Conv2d(128, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        self.linear_layer = nn.Sequential(
            nn.Linear(in_features=8192, out_features=256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(in_features=256, out_features=10),
        )

    def forward(self, x):
        x = self.convolutional_layer(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear_layer(x)
        return x
    
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

Load the corresponding data partition as [type, class, item] and transorm it into a PyTorch dataset:

In [428]:
data_file = open(os.path.join(data_path, 'partition_0.pkl'), 'rb')
data = pkl.load(data_file)

transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        tuples = [(image, label) for label in data.keys() for image in data[label]]
        random.shuffle(tuples)
        self.data, self.labels = zip(*tuples)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.transform(self.data[idx]) if self.transform else self.data[idx], self.labels[idx]

train_dataset = ImageDataset(data['train'], transform)
test_dataset = ImageDataset(data['test'], transform)

trainloader = DataLoader(train_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Train the model:

In [429]:
for epoch in range(10):
    running_loss = 0.0

    correct_predictions_train = 0
    total_samples_train = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted_train = torch.max(outputs, 1)
        total_samples_train += labels.size(0)
        correct_predictions_train += (predicted_train == labels).sum().item()

        running_loss += loss.item()

    accuracy_train = 100 * correct_predictions_train / total_samples_train

    correct_predictions_test = 0
    total_samples_test = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted_test = torch.max(outputs, 1)
            total_samples_test += labels.size(0)
            correct_predictions_test += (predicted_test == labels).sum().item()

    accuracy_test = 100 * correct_predictions_test / total_samples_test

    print(f'Epoch: {epoch + 1}. Loss: {running_loss / 100:.3f}. Train accuracy: {accuracy_train:.2f}%. Test accuracy: {accuracy_test:.2f}%')

    scheduler.step()
            
print('Finished Training')

Epoch: 1. Loss: 1.288. Train accuracy: 41.86%. Test accuracy: 51.40%
Epoch: 2. Loss: 0.962. Train accuracy: 56.86%. Test accuracy: 56.80%
Epoch: 3. Loss: 0.652. Train accuracy: 74.24%. Test accuracy: 62.30%
Epoch: 4. Loss: 0.525. Train accuracy: 80.82%. Test accuracy: 62.70%
Epoch: 5. Loss: 0.430. Train accuracy: 86.02%. Test accuracy: 64.00%
Epoch: 6. Loss: 0.413. Train accuracy: 87.18%. Test accuracy: 64.90%
Epoch: 7. Loss: 0.400. Train accuracy: 88.18%. Test accuracy: 64.90%
Epoch: 8. Loss: 0.399. Train accuracy: 88.22%. Test accuracy: 64.90%
Epoch: 9. Loss: 0.397. Train accuracy: 88.24%. Test accuracy: 64.90%
Epoch: 10. Loss: 0.397. Train accuracy: 88.24%. Test accuracy: 64.90%
Finished Training


Check the final accuracy on the test dataset:

In [430]:
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct / total} %')

Accuracy of the network on the 1000 test images: 64.9 %
